In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os, random

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

In [2]:
all_df = pd.read_csv('/content/drive/MyDrive/인연준/금융데이터분석/input/all_asset_2010_2020.csv')
all_df['Date'] = all_df.Date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
all_df.set_index('Date',inplace=True)

In [3]:
s_list = pd.read_csv('/content/drive/MyDrive/인연준/금융데이터분석/input/NASDAQ_stock_list.csv')

In [4]:
symbols = ['AAPL', 'MSFT', 'AMZN', 'FB']#, 'GOOGL', 'GOOG', 'TSLA', 'NVDA', 'CMCSA', 'ADBE']
st ='2020-06-01'
en = '2020-09-30'

In [84]:
import pandas_datareader.data as web
import statsmodels.api as sm

def cal_beta(asset_name):
  reg_df = ret_df.copy()[[asset_name, 'S&P500']].resample('M').last().pct_change().dropna(axis=0)
  Y = reg_df[asset_name]
  X = sm.add_constant(reg_df['S&P500'])
  model = sm.OLS(Y, X)
  results = model.fit()
  return results.params['S&P500']

def time_varying_beta(asset_name, alpha):
  reg_df = ret_df.copy()[[asset_name, 'S&P500']].resample('M').last().pct_change().dropna(axis=0)
  lambdas = [alpha**(i-1) for i in range(len(reg_df), 0, -1)]
  var = np.sum(lambdas*(reg_df['S&P500']**2)) * (1-alpha)
  cov = np.sum(reg_df['S&P500']*reg_df[asset_name]*lambdas) * (1-alpha)

  return cov/var


mpfo = web.get_data_yahoo(['^GSPC'], start='2010-01-01', end='2020-10-30')['Adj Close'].rename(columns={'^GSPC':'S&P500'})
topn = s_list.head(1000)['Symbol'].values
ret_df = all_df.copy()
ret_df['S&P500'] = mpfo['S&P500']
ret_df = ret_df.loc['2015-01-01':]

beta = dict()
for col in topn:
  try:
    # beta[col] = cal_beta(col)
    beta[col] = time_varying_beta(col, 0.975)
  except:
    pass

top_asset_beta = pd.DataFrame({'asset':beta.keys(),
                               'beta':beta.values()}).merge(s_list[['Symbol', 'Name']], left_on='asset', right_on='Symbol', how='left')[['Symbol', 'Name', 'beta']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


CPU times: user 24.4 s, sys: 239 ms, total: 24.6 s
Wall time: 24.9 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


In [83]:
top_asset_beta

,Symbol,Name,beta
0,AAPL,Apple Inc,1.455904
1,MSFT,Microsoft Corp,0.955114
2,AMZN,Amazon.com Inc,1.332300
3,FB,Facebook Inc Class A,1.395407
4,GOOGL,Alphabet Inc Class A,1.054661
...,...,...,...
995,RDNT,RadNet Inc,1.748667
996,FBNC,First Bancorp (North Carolina),0.978185
997,SRCE,1st Source Corp,1.047724
998,GRVY,Gravity Co Ltd ADR,0.125981
